In [2]:
from os import listdir
from os.path import isfile, join
import re
import csv
import gensim
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
import logging
import os
import pandas as pd 
import sys
import sklearn.metrics
import sklearn
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer
import random
import math
import warnings
import utils
from utils import *
from utils2 import *
import scipy.stats
from dataDictionariesLexicons import *
from testData import *
from polesConstructs import *

random.seed(1)

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
# warnings.filterwarnings('ignore')

sys.path.insert(0, '../') # necessary in order to import word2vecTraining where the epoch saver class, which is part of the model, is located

In [3]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
# modelFilePath = os.path.join(modelsPath, 'GoogleNews-vectors-negative300.bin')
# model2 = KeyedVectors.load_word2vec_format(modelFilePath,binary=True) 
modelFilePath = os.path.join(modelsPath, 'GoogleNews-vectors-negative300LowerCase.bin')
model1 = KeyedVectors.load(modelFilePath) 

In [4]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.6B.300d.txt.word2vec.bin')
model2 = KeyedVectors.load_word2vec_format(modelFilePath,binary=True) 

In [5]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.twitter.27B.200d.txt.word2vec.bin')
model3 = KeyedVectors.load(modelFilePath) 

In [6]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.42B.300d.txt.word2vec.bin')
model4 = KeyedVectors.load(modelFilePath) 

In [7]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.840B.300d.txt.word2vec.bin')
model5 = KeyedVectors.load(modelFilePath) 

In [8]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'fastext-common-crawl-300d-2M-subword.bin')
model6 = KeyedVectors.load(modelFilePath)

In [9]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'fastext-wiki-news-300d-1M-subword-Original.bin')
model7 = KeyedVectors.load(modelFilePath)

In [10]:
# models = [model1,model2,model3,model4]
# models = [model1,model2]
models = [model1,model2,model3,model4,model5,model6,model7]
# models = [model1]

In [11]:
#change default lexicons
# lexicons = [harvardGeneralInquirer3623, vaderLexicon7062,NRCEmotionLexicon5555,opinionLexicon6786,afinnLexicon2477]
# lexicons = [WEAT1]
lexicons = [harvardGeneralInquirer3623]
dataDict = dataDictLoadLexicons(dataDict,lexicons)
# WEAT1 harvardGeneralInquirer3623

In [12]:
dataSetIndexes = [
# '0-1', #percentage females per occupation
# '0-2', #country's GDP per capita
# '0-3', #car brands and prices
# '0-4', #voting behavior by ethnic/racial group
# '0-5', #voting behavior by profession
    
'1-1', #gender
'1-2',
'1-3',
'1-4',
'1-5',
    
'2-1', #Race/ethnicity
'2-2',
'2-3',
'2-4',    
  
'3-1', #Sexual orientation  
    
'4-1', #religiosity    
'4-2', #religion Islam and Christianity
    
'5-1', #age 
'6-1', #socioeconomic status
'7-1', #physical beauty
    
'8-1', #Personal ideology
'8-2', #Party affiliation and political parties
'8-3', #US presidents
'8-4', #Ideologies abstract
'8-5', #influential conservatives and liberals
]

In [13]:
rows = len(dataSetIndexes)
columns = len(models)
outputMatrix = np.zeros((rows,columns))
outputMatrixPvalues = np.zeros((rows,columns))

## Dimensions against models

In [14]:
#Checking for the existence of uppercase letters in models
# [w for w in model6.wv.vocab if any(x.isupper() for x in w)]

In [15]:
#Checking for the existence _ in models
# [w for w in model2.wv.vocab if '-' in w]

In [16]:
random.seed(0)
outputMatrices = []
for outputMatrixIndex in range(500):
    print("outputMatrixIndex",outputMatrixIndex)
    for rowNumber, dataSetIndex in enumerate(dataSetIndexes):
        print(dataDict[dataSetIndex]['name'])
        modelsCorrelations=[]
        modelspvalues=[]
        for i,model in enumerate(models):    
    #         print("model", i+1)
            correlations = []
            pvalues = []
            constructPole1 = dataDict[dataSetIndex]['constructPole1']
            constructPole2 = dataDict[dataSetIndex]['constructPole2']

            lengthPole1 = len(constructPole1)
            lengthPole2 = len(constructPole2)
            chunkSize=2
            if len(constructPole1) > 2:
                constructPole1 = random.sample(constructPole1,k=chunkSize*(max(1,round(lengthPole1/4))))
            else:
                if chunkSize <= 2:
                    constructPole1 = random.sample(constructPole1,k=1)
            if len(constructPole2) > 2:
                constructPole2 = random.sample(constructPole2,k=chunkSize*(max(1,round(lengthPole2/4))))
            else:
                if chunkSize <= 2:
                    constructPole2 = random.sample(constructPole2,k=1)
                
            if i>=1: #Glove and fastext vocabulary uses - instead of _ So we need to alter Construct poles and real data - Checked!
                constructPole1,constructPole2,RealDataTemp=replaceUnderscoreForDash(constructPole1,constructPole2,RealDataTemp)
            if i in [1,2,3]: # Lowercase poles for models With only lowercase vocabularies - Checked!
                constructPole1 = list(set(list(map(str.lower,constructPole1))))
                constructPole2 = list(set(list(map(str.lower,constructPole2))))
            constructPole1,constructPole2 = constructsFilter(model,constructPole1,constructPole2,printFlag=False)# Filter out terms not in model vocabulary

            for RealDataTemp in dataDict[dataSetIndex]['RealDataLexicons']:
    #         for RealDataTemp in [voteByEthnicity]:
                RealData = realDataFilter(model,RealDataTemp,0)#Filter out terms not in model vocabulary (Last parameter Flag prints out OOV words)
                Axis = dimensionN(model, constructPole1,constructPole2)  
                AxisName='Axis'
                try:
                    dataFrame = makeDF(model, RealData.keys(),Axis,AxisName)
                except:
                    continue
                dataFrame['RealValues']=RealData.values()
                spearmanCorr,pearsonCorr=calculateCorrelations(dataFrame,printFlag=False)
    #             correlations.append(pearsonCorr[0])
    #             pvalues.append(pearsonCorr[1])
                correlations.append(spearmanCorr[0])
                pvalues.append(spearmanCorr[1])            
    #         print("correlations for model", i+1)
    #         print(*correlations)
    #         print("mean: ", np.array(correlations).mean())
#             modelsCorrelations.append(np.array(correlations).mean())
            modelsCorrelations.append(np.nanmean(correlations))
#             modelspvalues.append(np.array(pvalues).mean())#This can only be done if only one lexicon is being tested
            modelspvalues.append(np.nanmean(pvalues))#This can only be done if only one lexicon is being tested
    #         print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        print(*modelsCorrelations)
        print(np.array(modelsCorrelations).mean())
        outputMatrix[rowNumber]=modelsCorrelations
        outputMatrixPvalues[rowNumber]=modelspvalues
        #Print closest neighbors to axis Poles
    #     print([w[0] for w in model.wv.similar_by_vector(-Axis,restrict_vocab=50000,topn=50)])
    #     print([w[0] for w in model.wv.similar_by_vector(Axis,restrict_vocab=50000,topn=50)])

        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    outputMatrices.append(outputMatrix)
# outputMatrix=np.array(outputMatrices).mean(axis=0) #shape of outputMatrices=(2,20,7)    
outputMatrix=np.nanmean(np.array(outputMatrices),axis=0) #shape of outputMatrices=(2,20,7)    

outputMatrixIndex 0
gender general
0.08866284883912301 0.29857731073734883 0.11967265485287414 0.18720695361648249 0.27778059101546787 0.17082822627819508 0.0314486633683135
0.16773960695825782
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender family roles
0.0006248521363190087 0.14704433112453386 -0.011829842124310305 0.14900026471679428 0.10258990839539037 0.27084902546835976 0.02035048765400319
0.09694700391015573
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and young age
0.06633262080220578 0.0895194092047702 0.13809349795979742 0.2515421016121563 0.23738130788484177 0.1903600394281935 0.11702840472383809
0.15575105451654328
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender masculinity and femininity
0.16366117511467126 0.2690850199989219 0.017514120601629497 0.3108495055162786 0.279058758614648 0.11675018926281552 0.12807417758286233
0.18357042095597528
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and popular given names 
0.036992157096635

0.11635507420493273 0.16167918990522157 0.023272544302540124 0.3636175839223808 -0.1514273410104318 0.3811887954178312 0.22095983918887382
0.15937795513304978
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
party affiliation and political parties
-0.04819664242415251 0.05803285218596192 -0.007337547165186298 -0.03893523908994379 0.09966769571224944 0.07913969099721055 -0.024178034835341013
0.0168846821972569
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
U.S. presidents
-0.03358479795486885 -0.1128839321559408 0.24271240844731673 0.2188956821263691 0.21905111328865967 0.06933080984220819 0.2082644458975463
0.1159693899273272
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
ideologies abstract


C:\Users\drozado\Sync\workspace\AlgorithmicBias\AlgorithmicBiasInWordEmbeddings\analysis\utils.py:15: RuntimeWarning: invalid value encountered in double_scalars
  normalized_vector = vector / np.linalg.norm(vector)
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: Mean of empty slice
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice
C:\Users\drozado\Sync\workspace\AlgorithmicBias\AlgorithmicBiasInWordEmbeddings\analysis\utils.py:15: RuntimeWarning: invalid value encountered in double_scalars
  normalized_vector = vector / np.linalg.norm(vector)
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: Mean of empty slice
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice


nan 0.13320114295244506 nan -0.16801093800974004 -0.04670145592741937 0.2424137334948537 0.014503826452445807
nan
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
influential conservatives and liberals
0.18830815024923686 0.16711633419503408 0.16770687386458855 0.2708502840370466 0.25144969751911067 0.0037969776431345512 0.14862465163611396
0.17112185273489505
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
outputMatrixIndex 2
gender general
0.05156007711227599 0.12611598724046516 -0.01469320614071554 0.18138122273296486 0.13259132748562144 0.23887059948888054 0.22080530424400005
0.13380447316621322
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender family roles
-0.06147575469021095 0.14825420215583915 0.060218373425858604 0.01249217877543872 0.0501097248159979 0.13763903821862739 0.22077535975641202
0.08114473177970898
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and young age
0.06633262080220578 0.11921675825858594 -0.054813696020708645 0.20415963286026773 -0

C:\Users\drozado\Sync\workspace\AlgorithmicBias\AlgorithmicBiasInWordEmbeddings\analysis\utils.py:15: RuntimeWarning: invalid value encountered in double_scalars
  normalized_vector = vector / np.linalg.norm(vector)
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: Mean of empty slice
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice


...

C:\Users\drozado\Sync\workspace\AlgorithmicBias\AlgorithmicBiasInWordEmbeddings\analysis\utils.py:15: RuntimeWarning: invalid value encountered in double_scalars
  normalized_vector = vector / np.linalg.norm(vector)
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: Mean of empty slice
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice


-0.09246445673554159 0.20554173066722373 nan -0.04663896298587986 0.33835441170925007 0.018812456049945872 -0.11992606862096994
nan
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
influential conservatives and liberals
0.1677204374281925 0.20912445791855425 0.14167567444918833 0.24529227333049863 0.31761209639518356 0.21079245029825772 -0.1066050494282502
0.16937319148451782
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
outputMatrixIndex 499
gender general
0.23067221446003813 0.03514069090707842 0.05335343131180857 0.048219980994059206 0.2044507324597467 0.20259791858337328 0.14758568487082785
0.1317172362267046
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender family roles
0.056497828502277435 0.18091371432267603 0.04854021771317048 0.16833248288843056 0.015033558398045929 0.13030559964094873 0.16875376397116298
0.10976816649095889
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and young age
0.18145861380123005 0.14918599147857536 -0.054813696020708645 0.204

C:\Users\drozado\Sync\workspace\AlgorithmicBias\AlgorithmicBiasInWordEmbeddings\analysis\utils.py:15: RuntimeWarning: invalid value encountered in double_scalars
  normalized_vector = vector / np.linalg.norm(vector)
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: Mean of empty slice
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice
C:\Users\drozado\Sync\workspace\AlgorithmicBias\AlgorithmicBiasInWordEmbeddings\analysis\utils.py:15: RuntimeWarning: invalid value encountered in double_scalars
  normalized_vector = vector / np.linalg.norm(vector)
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: Mean of empty slice
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice


0.44947170266670583 nan nan 0.279050746699066 0.13370979089241922 0.13344888320689266 0.3854240553376536
nan
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
influential conservatives and liberals
0.0729196818564906 0.2666809065410119 0.16410658556485466 0.2975369065852442 0.31909376665014627 0.04385142158891809 0.1517779131237291
0.18799531170148498
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:81: RuntimeWarning: Mean of empty slice


In [17]:
#print output matrix
for r in outputMatrix:
    print(*r)

0.23067221446003966 0.03514069090707871 0.053353431311808856 0.04821998099405895 0.20445073245974746 0.20259791858337484 0.14758568487082746
0.05649782850227701 0.1809137143226754 0.04854021771317084 0.16833248288843092 0.01503355839804588 0.13030559964094837 0.16875376397116348
0.18145861380122977 0.14918599147857495 -0.054813696020708826 0.20415963286026792 0.15679639270577628 0.1640985672034192 0.1170284047238392
0.14787997598315278 0.2690850199989231 0.017514120601629497 -0.028647405354963244 0.2805206661445092 0.03666692596553599 0.12807417758286158
0.02605711079573514 -0.11489733002161114 0.013957413845533383 0.05455259907006395 -0.013146485214634572 0.2574740549416675 0.02324013070624532
-0.08955419384964106 0.061313246406606396 0.003041652149468649 -0.08573764515369678 0.03828253503386462 0.019131161941397676 -0.11217739773458235
-0.03737087159329156 -0.14617775087735185 -0.06723273797523824 -0.2228498295583289 -0.21893467346281562 -0.15656226725429592 -0.0677889726693097
-0.01

In [18]:
#print P values output matrix
for r in outputMatrixPvalues:
    print(*r)

9.284890629752158e-45 0.035126707256032334 0.0015738440737468763 0.0037519345373897995 2.2257806025171183e-35 9.095119763912449e-35 5.013191444532196e-19
0.0006882227649504627 7.940902447777423e-28 0.004037482102446745 2.3226526849212128e-24 0.3664575454987402 3.774846724569308e-15 1.832487899803608e-24
4.556909803587915e-28 2.4234339363713453e-19 0.0011648610987160491 2.788495508724444e-35 2.602987350215517e-21 3.2005034648301195e-23 1.7600371218141193e-12
4.407831324872819e-19 1.123702097568187e-60 0.2997849063448032 0.08520990168801394 2.7702904503464064e-66 0.027548443961496416 1.13436537796822e-14
0.11771143740399431 4.862007220290028e-12 0.4086324328250268 0.0010400598685516875 0.4296715049062825 8.729681831889723e-56 0.16275827474681498
7.176750039979003e-08 0.0002346817219977543 0.857103401504154 2.471724986132772e-07 0.021419600779107102 0.250354619767148 1.3985456932537932e-11
0.02482430264982751 1.2640366783184414e-18 6.759174941236528e-05 7.143595834003511e-42 1.93451385003

## adjusting P values for multiple tests

In [19]:
outputMatrixPvalues.shape

(20, 7)

In [20]:
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

stats = importr('stats')

#bonferroni most conservative, holm also valid under arbitrary assumptions
p_adjust = stats.p_adjust(FloatVector(outputMatrixPvalues.flatten()), method = 'holm')
outputMatrixPvaluesCorrected = np.array(p_adjust).reshape(outputMatrixPvalues.shape)

In [21]:
#print P values output matrix
for r in outputMatrixPvaluesCorrected:
    print(*r)

1.0027681880132331e-42 0.7727875596327114 0.05036301035990004 0.11630997065908379 2.203522796491947e-33 8.822266170995077e-33 3.8100254978444686e-17
0.024087796773266196 7.067403178521906e-26 0.12112446307340234 1.997481309032243e-22 1.0 2.5291473054614365e-13 1.594264472829139e-22
4.1012188232291236e-26 1.8902784703696492e-17 0.038440416257629624 2.732725598549955e-33 2.134449627176724e-19 2.7204279451056016e-21 1.0208215306521892e-10
3.3940301201520705e-17 1.3259684751304606e-58 1.0 1.0 3.352051444919152e-64 0.6336142111144176 7.37337495679343e-13
1.0 2.771344115565316e-10 1.0 0.035362035530757376 1.0 9.777243651716489e-54 1.0
3.5166075195897117e-06 0.008917905435914662 1.0 1.1617107434824028e-05 0.5354900194776776 1.0 7.692001312895863e-10
0.5957832635958602 9.48027508738831e-17 0.0027712617259069766 7.500775625703687e-40 2.0118944040336377e-38 2.3882863813760903e-19 0.0019268196455039235
1.0 0.22909971033903315 1.0 1.0 6.565421817009861e-08 5.717302233149461e-31 5.0972680901895057e